# Code to use all best models for 5 folds and print confusion matrix and some inferred images and save!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
import random
import copy
from tqdm import tqdm
tqdm.pandas()
import gc
from collections import defaultdict
import time
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
from torch.optim import lr_scheduler
import cv2
import matplotlib
matplotlib.style.use('ggplot')
import albumentations as A
from albumentations.pytorch import ToTensorV2
Image.MAX_IMAGE_PIXELS = None
pd.set_option('display.float_format', '{:.2f}'.format)
import segmentation_models_pytorch as smp
from torchvision.models.resnet import Bottleneck, ResNet
import pickle
from natsort import natsorted
from sklearn.metrics import f1_score

In [ ]:
class model_config:
    start_fold = 1  # start CV fold to train
    total_fold = 5  # total # of folds
    key = "BT"  # key of resnet model to train if pretrained_resnet = True
    pretrained_resnet = False  # whether to train using a pretrained resnet model
    seed = 42  # random seed
    train_batch_size = 4
    valid_batch_size = 8
    epochs = 300
    patience = int(epochs / 10)
    learning_rate = 0.002  # 0.001 for bs=16
    scheduler = "CosineAnnealingLR"  # explore different lr schedulers, with cosineannealingLR, at T_max, the learning rate will be about half of initial lr above.
    num_training_samples = 160
    T_max = max(10, int(
        num_training_samples / train_batch_size * epochs))  # number of iterations for a full cycle, need to change for different # of iterations (iteration = batch size). low BS and num_train sample -> low T_max.
    weight_decay = 1e-6  # explore different weight decay (for Adam optimizer)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    iters_to_accumulate = max(1, 32 // train_batch_size)  # for scaling accumulated gradients, should never be <1
    eta_min = 1e-5
    model_save_directory = os.path.join(os.getcwd(), "model",
                                        "DeepLabV3+_baseline")  # assuming os.getcwd is the current training script directory


# %%
# sets the seed of the entire notebook so results are the same every time we run for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)  # numpy specific random
    random.seed(seed)  # python specific random (also for albumentation augmentations)
    torch.manual_seed(seed)  # torch specific random
    torch.cuda.manual_seed(seed)  # cuda specific random
    # when running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # when deterministic = true, benchmark = False, otherwise might not be deterministic

    os.environ['PYTHONHASHSEED'] = str(seed)  # set a fixed value for the hash seed, for hashes like dictionary


set_seed(model_config.seed)  # set seed first

In [ ]:
val_transforms = A.Compose([ToTensorV2()])


In [ ]:
class TrainDataSet(Dataset):
    # initialize df, label, imagepath, transforms:
    def __init__(self, image_paths: list, mask_paths: list, transforms=None, label=True):
        self.label = label
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    # define main function to read image and label, apply transform function and return the transformed images.
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        image = np.array(image)
        if self.label:
            mask_path = self.mask_paths[idx]
            mask = cv2.imread(mask_path, 0)
            mask = np.array(mask)
            mask[mask == 13] = 0
        if self.transforms is not None:  # albumentations
            transformed = self.transforms(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']
            mask = mask.unsqueeze(0)  # dtypes: image = torch.uin8, mask = torch.uint8
        return image, mask  # return tensors of equal dtype and size
        # image is size 3x1024x1024 and mask is size 1x1024x1024 (need dummy dimension to match dimension)


# %%
# define dataloading function to use above dataset to return train and val dataloaders:
def load_dataset():
    train_dataset = TrainDataSet(image_paths=training_images, mask_paths=training_labels, transforms=train_transforms)
    val_dataset = TrainDataSet(image_paths=val_images, mask_paths=val_labels, transforms=val_transforms)
    train_dataloader = DataLoader(dataset=train_dataset,
                                  batch_size=model_config.train_batch_size,
                                  # pin_memory= true allows faster data transport from cpu to gpu
                                  num_workers=0, pin_memory=True, shuffle=False)
    val_dataloader = DataLoader(dataset=val_dataset,
                                batch_size=model_config.valid_batch_size,
                                num_workers=0, pin_memory=True, shuffle=False)
    return train_dataloader, val_dataloader  # return train and val dataloaders


# %%
# test to see if dataloaders return desired batch size and visualize images to see if images are indeed transformed:
train_dataloader, val_dataloader = load_dataset()
images, labels = next(iter(train_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(images.size(), labels.size()))
images, labels = next(iter(val_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(images.size(), labels.size()))


# %%
def visualize_images(dataset: torch.utils.data.Dataset, num_images: int):
    indices = random.sample(range(len(dataset)), num_images)
    fig, axes = plt.subplots(nrows=num_images, ncols=2, figsize=(10, 12))
    fig.tight_layout()
    for i, ax_row in enumerate(axes):
        index = indices[i]
        image, mask = dataset[index]
        if dataset.transforms is not None:
            ax_row[0].imshow(image.permute(1, 2, 0))
        else:
            ax_row[0].imshow(image)

        ax_row[0].set_title("Image")
        ax_row[0].axis("off")

        if dataset.transforms is not None:
            ax_row[1].imshow(mask.squeeze(0), cmap="gray")
        else:
            ax_row[1].imshow(mask, cmap="gray")
        ax_row[1].set_title("Mask")
        ax_row[1].axis("off")
    plt.show()


visualize = True  # always check if transforms properly applied before training
if visualize:
    original_dataset = TrainDataSet(image_paths=training_images, mask_paths=training_labels, transforms=None)
    visualize_images(original_dataset, num_images=3)
    train_dataset = TrainDataSet(image_paths=training_images, mask_paths=training_labels, transforms=train_transforms)
    visualize_images(train_dataset, num_images=3)


In [ ]:
def build_model():
    if model_config.pretrained_resnet:  # only pretrained resnet50 available
        model = smp.DeepLabV3Plus(encoder_name="resnet50", encoder_weights=model_config.key, encoder_depth=5,
                                  decoder_channels=512, activation=None, in_channels=3, classes=13)
    else:  # try different encoders
        model = smp.DeepLabV3Plus(encoder_name="resnet50", encoder_weights="imagenet", encoder_depth=5,
                                  decoder_channels=512, activation=None,
                                  in_channels=3, classes=13)
    model.to(model_config.device)  # move model to gpu
    return model

In [ ]:
def calculate_f1_score(y_pred: torch.Tensor, y_true: torch.Tensor):  # y_pred in probabilities
    y_pred = y_pred.cpu().numpy().flatten()  # 1d numpy array
    y_true = y_true.cpu().numpy().flatten()  # 1d numpy array
    return f1_score(y_pred, y_true,
                    average="weighted")

In [ ]:
# finally run training:
data_src = r"C:\Users\Kevin\Desktop\deeplab_trainingset"
start_fold = model_config.start_fold

# repeat for all number of folds:
for fold in range(0, model_config.total_fold):
    current_fold = start_fold + fold
    fold_dir = r"v11_fold{}".format(current_fold)
    image_path = os.path.join(data_src, fold_dir)
    train_dir = os.path.join(image_path, "training")
    val_dir = os.path.join(image_path, "validation")

    train_image_dir = os.path.join(train_dir, "im")
    train_label_dir = os.path.join(train_dir, 'label')
    training_images = natsorted(
        [os.path.join(train_image_dir, x) for x in os.listdir(train_image_dir) if x.endswith(".png")])
    training_labels = natsorted(
        [os.path.join(train_label_dir, x) for x in os.listdir(train_label_dir) if x.endswith(".png")])

    val_image_dir = os.path.join(val_dir, "im")
    val_label_dir = os.path.join(val_dir, 'label')
    val_images = natsorted([os.path.join(val_image_dir, x) for x in os.listdir(val_image_dir) if x.endswith(".png")])
    train_dataloader, valid_dataloader = load_dataset()  # load datasets
    model = build_model()  # build model
    # print(model)
    optimizer = optim.Adam(model.parameters(),
                           lr=model_config.learning_rate,
                           weight_decay=model_config.weight_decay)  # initialize optimizer
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer,
                                               T_max=model_config.T_max,
                                               eta_min=model_config.eta_min)  # initialize LR scheduler
    print("Training for Fold {}".format(current_fold))